# Extract, Transform, and Load - Genius API

### Data used:
* musicdf: from Ashley's EDA & https://www.kaggle.com/datasets/josephinelsy/spotify-top-hit-playlist-2010-2022
* unpopular_df: from Michelle's EDA & https://www.kaggle.com/datasets/estienneggx/spotify-unpopular-songs?resource=download
* Lyrics (text) from Genius: https://docs.genius.com/#songs-h2

### Imported Dependencies
* pathlib, pandas, os for manipulating data
* requests for API calls
* lyricsgenius from https://pypi.org/project/lyricsgenius/ to assist with Genius API access, scraping, other functions

### Additional Pre-processing
* sign up for Genius account for API access
* obtain client_id and client_secret
* retrieve access_token using client_id and client_secret
* save access_token and use for API calls

### Post-processing
* configuring gitignore file to house api keys
* additional API calls (Genius sets a daily limit)


Setup: importing dependencies, configure lyricsgenius package parameters, import access token

In [ ]:
# import dependencies
from pathlib import Path
import pandas as pd
import os
import requests
import lyricsgenius as lg
from lyricsgenius import Genius, OAuth2

import warnings
warnings.filterwarnings("ignore")

client_id, redirect_uri, client_secret = lg.auth_from_environment()

In [ ]:
# API access tokens
from api_keys import access_token
genius = Genius(access_token)

In [ ]:
# configure package parameters

# increase timeout value
genius.timeout = 120

# turn off status messages
genius.verbose = False

# remove section headers (e.g. [chorus]) from lyrics when searching
genius.remove_section_headers = True

# include hits thought to be non-songs (e.g. track lists)
genius.skip_non_songs = False

# exclude songs with "remix" and "live" in their title
genius.excluded_terms = ["(Remix)", "(Live)"]

Popular Songs - scraping & appending lyrics to new dataframe

In [76]:
# check working directory
os.chdir("C:/Users/yiran/OneDrive/Documents/GitHub")

# import CSV files
file_path = Path('C:/Users/yiran/OneDrive/Documents/GitHub/project-4/Zhang - Project 4/musicdf.csv')
popular_df = pd.read_csv(file_path)
popular_df.head()

,year,track_name,track_popularity,album,artist_name,artist_genres,artist_popularity,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_min
0,2019,Cruel Summer,96,Lover,Taylor Swift,['pop'],100,0.552,0.702,9,-5.707,0.1570,0.1170,0.000021,0.1050,0.564,169.994,2.97
1,2023,Cruel Summer,96,Lover,Taylor Swift,['pop'],100,0.552,0.702,9,-5.707,0.1570,0.1170,0.000021,0.1050,0.564,169.994,2.97
2,2023,greedy,96,greedy,Tate McRae,['pop'],85,0.750,0.733,6,-3.180,0.0319,0.2560,0.000000,0.1140,0.844,111.018,2.20
3,2023,My Love Mine All Mine,94,The Land Is Inhospitable and So Are We,Mitski,"['brooklyn indie', 'pov: indie']",84,0.504,0.308,9,-14.958,0.0321,0.8680,0.135000,0.1580,0.121,113.950,2.30
4,2023,Lovin On Me,93,Lovin On Me,Jack Harlow,"['deep underground hip hop', 'kentucky hip hop...",83,0.943,0.558,2,-4.911,0.0568,0.0026,0.000002,0.0937,0.606,104.983,2.31


In [ ]:
# create dataframe to contain song names, artists, and lyrics
lyrics_df = pd.DataFrame(columns=['track_name',
                               'track_popularity',
                               'album',
                               'artist_name',
                               'artist_genres',
                               'artist_popularity',
                               'lyrics'])

In [ ]:
# check dataframe setup
lyrics_df.head()

,track_name,track_popularity,album,artist_name,artist_genres,artist popularity,lyrics


In [ ]:
# iterate through popular_df to retrieve lyrics and append to lyrics_df 
for i in range(len(popular_df)):
    song = genius.search_song(popular_df['track_name'][i], popular_df['artist_name'][i])
    if song:
        new_row = pd.Series({'track_name': popular_df['track_name'][i],
                             'track_popularity': popular_df['track_popularity'][i],
                             'album': popular_df['album'][i],
                             'artist_name': popular_df['artist_name'][i],
                             'artist_genres': popular_df['artist_genres'][i],
                             'artist_popularity': popular_df['artist_popularity'][i],
                             'lyrics': song.lyrics})
    else:
        new_row = pd.Series({'track_name': popular_df['track_name'][i],
                             'track_popularity': popular_df['track_popularity'][i],
                             'album': popular_df['album'][i],
                             'artist_name': popular_df['artist_name'][i],
                             'artist_genres': popular_df['artist_genres'][i],
                             'artist_popularity': popular_df['artist_popularity'][i],
                             'lyrics': None})
    lyrics_df = pd.concat([lyrics_df, new_row.to_frame().T], ignore_index=True)

This iteration process took many tries across several days due to timeout errors and retry errors. The initial package parameters set timeout to 10, which allowed me to make 100 searches at a time. At first, I adjusted my process by batching the calls; i.e., "for i in range(200)" and then "for i in range(200,400)" and so on. This process was a crude way of appending each new batch to the end of the previous batch, but took a lot of time and monitoring. And, the timeout error occurred at random intervals--sometimes a range of 250 worked, sometimes it didn't. Later, I was able to fix this issue by increasing the timeout value to 60 and then 120. I was finally able to make over 1000 searches without running into the timeout error.

In addition, the runtimes were lengthy. For example:
* retrieving 100 songs' lyrics: 7 minutes 7 seconds
* 200 songs: 18 minutes, then 2nd attempt in 16 minutes
* 300 songs: 23 minutes
* 600 songs: 43 minutes
and so on

In [104]:
# check dataframe
lyrics_df

,track_name,track_popularity,album,artist_name,artist_genres,artist popularity,lyrics
0,Cruel Summer,96,Lover,Taylor Swift,['pop'],100,"(Yeah, yeah, yeah, yeah)\n\nFever dream high i..."
1,Cruel Summer,96,Lover,Taylor Swift,['pop'],100,"(Yeah, yeah, yeah, yeah)\n\nFever dream high i..."
2,greedy,96,greedy,Tate McRae,['pop'],85,"(Woo)\n\nHe said, ""Are you serious? I've tried..."
3,My Love Mine All Mine,94,The Land Is Inhospitable and So Are We,Mitski,"['brooklyn indie', 'pov: indie']",84,"Moon, a hole of light\nThrough the big top ten..."
4,Lovin On Me,93,Lovin On Me,Jack Harlow,"['deep underground hip hop', 'kentucky hip hop...",83,I don't like no whips and chains and you can't...
...,...,...,...,...,...,...,...
2395,Butter,0,Butter,BTS,"['k-pop', 'k-pop boy group', 'pop']",87,Smooth like butter\nLike a criminal undercover...
2396,Solar Power,0,Solar Power,Lorde,"['art pop', 'metropopolis', 'nz pop', 'pop']",76,"I hate the winter, can't stand the cold\nI ten..."
2397,Lost Cause,0,Lost Cause,Billie Eilish,"['art pop', 'pop']",89,Something's in the—\n\nSomething's in the air ...
2398,What You Know Bout Love,0,LOVE,Pop Smoke,"['brooklyn drill', 'rap']",80,Uh\n\nShawty go joggin' every morning (Every m...


In [105]:
# export lyrics_df to CSV
lyrics_df.to_csv('C:/Users/yiran/OneDrive/Documents/GitHub/project-4/Zhang - Project 4/lyricsdf.csv', index=False)

Unpopular Songs - scraping & appending lyrics to new dataframe

The goal for this investigation was to scrape lyrics for unpopular songs. Since the dataset was so large (4000+ songs) and included songs without any lyrics, I decided to implement a cutoff point to ensure my computer wouldn't crash or take an inordinate amount of time. In studying the popularity score distribution of the unpopular songs dataset, I saw that dropping all the songs with a popularity score of 0 would significantly improve my runtime and resulting analysis.

In [106]:
# import CSV files
file_path = Path('C:/Users/yiran/OneDrive/Documents/GitHub/project-4/Zhang - Project 4/unpopular_df_file.csv')
unpopular_df = pd.read_csv(file_path)
unpopular_df.head()

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_min,popularity,track_name,artist_name
0,0.722,0.476,5,-8.203,0.0896,0.6700,0.0000,0.0926,0.9270,151.968,3.68,8,Este Corazón,El Javi
1,0.955,0.281,0,-11.392,0.4660,0.0048,0.0595,0.1220,0.0934,127.991,2.49,0,thinking and doing,Br80
2,0.753,0.355,8,-11.799,0.5140,0.5730,0.0000,0.0735,0.6000,91.978,2.17,0,Puzzlement,JpThaRuler
3,0.650,0.331,3,-10.818,0.3830,0.8650,0.0000,0.1050,0.5810,99.513,1.94,0,Sensi,JpThaRuler
4,0.690,0.360,4,-15.624,0.4550,0.7640,0.0000,0.1810,0.1050,89.893,1.86,0,Rass,JpThaRuler


In [108]:
# drop rows with value of 0 in popularity column
unpopular_df = unpopular_df[unpopular_df['popularity'] != 0]
unpopular_df

,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_min,popularity,track_name,artist_name
0,0.722,0.476,5,-8.203,0.0896,0.6700,0.000000,0.0926,0.927,151.968,3.68,8,Este Corazón,El Javi
52,0.488,0.343,1,-14.357,0.4630,0.8010,0.000000,0.0677,0.517,81.262,3.72,2,Anybody,Blockos
53,0.730,0.787,1,-6.858,0.5970,0.0300,0.158000,0.2540,0.743,159.974,1.40,1,napalm,AC3!
56,0.703,0.692,8,-10.325,0.0858,0.0148,0.725000,0.0610,0.362,169.040,2.20,1,Luxury,Metzger Beats
58,0.713,0.558,1,-8.674,0.4510,0.2210,0.000000,0.1450,0.545,98.447,1.84,1,NAH,Yonny
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3973,0.588,0.416,7,-10.225,0.3930,0.5960,0.000000,0.1580,0.550,150.124,3.40,7,Blue Boy Benjamin Freestyle,Lilgrouch
3974,0.617,0.655,5,-9.124,0.2720,0.1830,0.000000,0.0541,0.944,197.239,5.26,1,El Sueno de Julian,Gilberto Perez
3976,0.465,0.338,8,-9.142,0.0254,0.5780,0.000000,0.0941,0.483,90.057,3.07,2,Mi Casa Nueva,Gilberto Perez
3978,0.835,0.433,10,-11.160,0.0615,0.4900,0.000003,0.0425,0.966,114.404,2.32,4,El Burro Pardo,Gilberto Perez


In [118]:
# reset index in new unpopular_df
unpopular_df = unpopular_df.reset_index(drop=True)

In [119]:
# create dataframe to store unpopular lyrics
unpop_lyrics_df = pd.DataFrame(columns=['track_name',
                               'popularity',
                               'artist_name',
                               'lyrics'])

In [ ]:
# check df
unpop_lyrics_df

,track_name,popularity,artist_name,lyrics


In [ ]:
# iterate through unpopular_df to retrieve lyrics and append to unpop_lyrics_df
for i in range(len(unpopular_df)):
    unpop_song = genius.search_song(unpopular_df['track_name'][i], unpopular_df['artist_name'][i])
    if unpop_song:
        new_row = pd.Series({'track_name': unpopular_df['track_name'][i],
                             'popularity': unpopular_df['popularity'][i],
                             'artist_name': unpopular_df['artist_name'][i],
                             'lyrics': unpop_song.lyrics})
    else:
        new_row = pd.Series({'track_name': unpopular_df['track_name'][i],
                             'popularity': unpopular_df['popularity'][i],
                             'artist_name': unpopular_df['artist_name'][i],
                             'lyrics': None})
    unpop_lyrics_df = pd.concat([unpop_lyrics_df, new_row.to_frame().T], ignore_index=True)

In [125]:
# check df
unpop_lyrics_df

,track_name,popularity,artist_name,lyrics
0,Este Corazón,8,El Javi,None
1,Este Corazón,8,El Javi,None
2,Este Corazón,8,El Javi,None
3,Anybody,2,Blockos,"(Doc)\nYeah, we in the spot all G'd up\nL-O-C'..."
4,napalm,1,AC3!,HO HO HO HO HO !\nMerry Christmas !\nOlá Pai N...
...,...,...,...,...
74,"Freestyle, Pt. 1",5,Dino BTW,None
75,"Freestyle, Pt. 2",5,Dino BTW,None
76,There Was Jesus,8,Jared Dixon,None
77,Chewbacca,2,Hosty,None


This is where I kept running into the "cap" on API calls that Genius implemented. I was unable to create a dataframe with more than 86 "unpopular" song lyrics, and for many of the songs in the unpopular_df, there were no lyrics on Genius.com in the first place. Unfortunately, this attempt was unsuccessful due to Genius limiting the number of API calls from my access token. Additionally, many unpopular songs had no lyrics on the website, and I kept running into the "cap" placed on "retries".

# Extract, Transform, Load - Spotify API

In this section, I attempted to load Spotify's 2024 song attributes data. This attempt was ultimately unsuccessful because I discovered that after November 27, 2024, Spotify deprecated their song attributes (audio features) for new API users. This meant that I would only be able to download the general artist information, such as name and popularity, as well as general song information, such as track popularity, but no other attributes for analysis. More info here: https://developer.spotify.com/blog/2024-11-27-changes-to-the-web-api

However, the code I wrote is still good, and I remain hopeful that at some point in the near future, Spotify may make those features available to programmers once again, and it'll be helpful to anyone hoping to extract audio features.

In [ ]:
# import dependencies
import numpy as np
import requests
from time import sleep

In [ ]:
# Get access token from the spotify API
response = requests.post('https://accounts.spotify.com/api/token', data={'grant_type': 'client_credentials'}, auth=(spotify_client_id, spotify_client_secret))
access_token = response.json()['access_token']
access_token

In [ ]:
# Set up the headers for the HTTP GET request
headers = {
    'Authorization': f'Bearer {access_token}',
    'Content-Type': 'application/json',
}

In [ ]:
# define url
urls = ['https://open.spotify.com/playlist/774kUuKDzLa8ieaSmi8IfS']

In [ ]:
# creating playlist dataframe
playlist_api_2024 = []
year = [2024]
ids = [url.split("/")[-1].split("?")[0] for url in urls]
api = 'https://api.spotify.com/v1/playlists/' + ids[0] + '/tracks'
playlist_api_2024.append(api)
playlist_api_2024 = pd.Series(playlist_api_2024, index = year)

In [ ]:
# create dataframe to hold top hits (2024)
TopHits2024 = pd.DataFrame(columns=['year',
                                'track_name',
                                'track_popularity',
                                'album',
                                'artist_name',
                                'artist_genres',
                                'artist_popularity',
                                'danceability',
                                'energy',
                                'key',
                                'loudness',
                                'speechiness',
                                'acousticness',
                                'instrumentalness',
                                'liveness',
                                'valence',
                                'tempo',
                                'duration_ms'])

In [ ]:
# loop through each track, retrieve audio features (song attributes), append to df
for playlist_api in playlist_api_2024:
    response = requests.get(playlist_api, headers=headers)
    top_tracks = response.json()['items']
    playlist_url = 'https://open.spotify.com/playlist/' + playlist_api.split("/")[-2].split("?")[0]
    year = '2024'
    for top_track in top_tracks:        
        track_id = top_track['track']['uri'].replace('spotify:track:', '')
        artist_id = top_track['track']['artists'][0]['uri'].replace('spotify:artist:', '')
        artist_info = requests.get('https://api.spotify.com/v1/' + 'artists' + '/' + artist_id, headers=headers)
        artist_info = artist_info.json()
        audio_features = requests.get('https://api.spotify.com/v1/' + 'audio-features' + '/' + track_id, headers=headers)

        if audio_features.status_code == 403:
            new_row = pd.Series({'track_id': track_id,
                   'track_name': top_track['track']['name'],
                   'track_popularity': top_track['track']['popularity'],
                   'album': top_track['track']['album']['name'],
                   'artist_name': top_track['track']['artists'][0]['name'],
                   'artist_genres': artist_info['genres'],
                   'artist_popularity': artist_info['popularity']})
        
        else:
            audio_features = audio_features.json()
            new_row = pd.Series({'year': year,
                                 'track_id': track_id,
                                 'track_name': top_track['track']['name'],
                                 'track_popularity': top_track['track']['popularity'],
                                 'album': top_track['track']['album']['name'],
                                 'artist_name': top_track['track']['artists'][0]['name'],
                                 'artist_genres': artist_info['genres'],
                                 'artist_popularity': artist_info['popularity'],
                                 'danceability': audio_features['danceability'],
                                 'energy': audio_features['energy'],
                                 'key': audio_features['key'],
                                 'loudness': audio_features['loudness'],
                                 'speechiness': audio_features['speechiness'],
                                 'acousticness': audio_features['acousticness'],
                                 'instrumentalness': audio_features['instrumentalness'],
                                 'liveness': audio_features['liveness'],
                                 'valence': audio_features['valence'],
                                 'tempo': audio_features['tempo'],
                                 'duration_ms': audio_features['duration_ms']})
        TopHits2024 = pd.concat([TopHits2024, new_row.to_frame().T], ignore_index=True)
        sleep(0.2)
    values = {'playlist_url': playlist_url, 'year': year}
    TopHits2024 = TopHits2024.fillna(value=values)

In [ ]:
# checking
# audio_features = requests.get('https://api.spotify.com/v1/' + 'audio-features' + '/' + '774kUuKDzLa8ieaSmi8IfS', headers=headers)

In [ ]:
# checking TopHits2024 dataframe progress
TopHits2024

In [ ]:
# export file to CSV
TopHits2024.to_csv('playlist_2024',index=False)